In [ ]:
import pandas as pd
from datetime import datetime
import gspread
import os

In [ ]:
def is_notebook() -> bool:
    try:
        shell = get_ipython().__class__.__name__
        if shell == 'ZMQInteractiveShell':
            #print("Jupyter notebook or qtconsole")
            return True
        elif shell == 'TerminalInteractiveShell':
            #print("Terminal running IPython")
            return False
        else:
            return False
    except NameError:
        return False

In [ ]:
CREDS_FILE = 'service_account.json'
if is_notebook():
    import IPython
    HTML = IPython.display.HTML
    creds_file_path = CREDS_FILE
else:
    creds_file_path = os.path.dirname(os.path.realpath(__file__)) + '/' +  CREDS_FILE

In [ ]:
gc = gspread.service_account(filename=creds_file_path)
sh = gc.open(str('transactions'))

worksheet = sh.worksheet('raw')
rdf = pd.DataFrame(worksheet.get_all_records())

In [ ]:
report_sh = gc.open('Report')
report_id = 'all'

try:
    report_worksheet = report_sh.add_worksheet(title=report_id, rows=200, cols=14)
except:
    report_worksheet = report_sh.worksheet(report_id)

In [ ]:
rdf['Date'] =  pd.DatetimeIndex(rdf.Date)
#rdf = rdf.drop(columns=["Notes", "Description", "Original Description", "Labels", "Notes"])

In [ ]:
debits = rdf[rdf['Transaction Type'] == 'debit']

In [ ]:
debits['Account Name'].unique()

In [ ]:
accounts = [
    'Credit (A)',
    'Credit (U4)',
    'Credit (U3)',
    'Credit (U2)',
    'Credit (U1)',
    'Checking (C)',
    'Checking (U)',
    'Checking (E)',
    'Savings (C)',
    'Savings (U)',
    'Savings (E)',
    'Savings (A)',
    'Cash',
    'HSA (F)',
]

debits = debits[debits['Account Name'].isin(accounts)]

In [ ]:
debits.Category.unique()


In [ ]:
ignored = [
    'Transfer',
    'Investments',
    'Buy',
    'Investment Account',
    'Buy',
    'Credit Card Payment',
    'Dividends & Cap Gains',
    'Transfer to NRE',
    'Allowance',
    'Withdrawal',
    'Option Exercise Cost'
]

In [ ]:
df = debits[~debits.Category.isin(ignored)]
ydf = df.groupby([df.Date.dt.to_period('Y'), "Category"])['Amount'].sum().reset_index(name='Amount').sort_values(by=['Date', 'Amount'], ascending=False)

In [ ]:
category_report = ydf.groupby(["Category"])['Amount'].sum().reset_index(name='Amount').sort_values(by=['Amount'], ascending=False)

pv_table = pd.pivot_table(ydf, index = 'Category', columns = 'Date', values = 'Amount', fill_value=0, dropna=False)


annual_monthly_report=category_report.merge(pv_table, on='Category')

annual_monthly_report.loc['Total'] = annual_monthly_report.sum(numeric_only=True)
annual_monthly_report = annual_monthly_report.fillna(value={"Category": "Total"}, limit=1).sort_values(by=['Amount'], ascending=False)

columns = annual_monthly_report.columns.values.tolist()
for idx, val in enumerate(columns):
    if isinstance(val, pd.Period):
        # pd.Period is not json serializable for writing to Google Sheets
        columns[idx] = val.strftime('%Y')
        
report_worksheet.update( [columns] + annual_monthly_report.values.tolist())
        